In [1]:


# Install simalign + scipy + pandas
!pip install simalign==1.0.8    # or latest simalign
!pip install pandas scipy tqdm


ERROR: Could not find a version that satisfies the requirement simalign==1.0.8 (from versions: 0.3, 0.4)
ERROR: No matching distribution found for simalign==1.0.8


In [2]:
from google.colab import files
uploaded = files.upload()

import zipfile, os

zip_path = "/content/conllu_file.zip"  # adjust if you used a different name
extract_path = "/content/conllu"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Files extracted to:", extract_path)



Saving conllu_file.zip to conllu_file.zip
✅ Files extracted to: /content/conllu


In [3]:
import os
import csv
from collections import defaultdict, Counter

# ---------- CONFIG ----------
CONLLU_ROOT = "conllu/conllu_file"
OUT_CSV = "conllu/adjectives_appraisal_with_examples_by_Domain_thresholded.csv"
KEEP_TOP_N = 20            # top-N adjectives per (lang, Domain) by total count
EXAMPLES_PER_CLASS = 5     # how many example sentences to keep per classification
CONFIDENCE_THRESHOLD = 0.50  # set to 0.50 or 0.60 as desired
# ---------- end CONFIG ----------

# ---- (keep your pattern sets unchanged) ----
FIRST_PERSON = {
    "i", "me", "we", "us",
    "my", "our",
    "mine", "ours",
    "myself", "ourselves"
}
SECOND_PERSON = {"you", "your", "yours", "yourself", "yourselves"}
THIRD_PERSON_PRONOUNS = {"he", "she", "it", "they", "him", "her", "them", "his", "hers", "theirs"}
APPRECIATION_PRONOUNS = {
    "it", "there", "here", "this", "that", "these", "those",
    "something", "anything", "nothing", "everything",
    "someone", "anyone", "no one", "everyone",
    "somebody", "anybody", "nobody", "everybody",
    "all", "each", "both", "neither", "none", "one",
    "what", "whatever", "which", "whichever",
    "this one", "that one", "such", "so"
}
FEEL_VERBS = {
    "be", "become", "get", "remain", "stay", "keep", "prove", "turn",
    "seem", "appear", "sound", "look", "smell", "taste", "feel",
    "think", "consider", "believe", "find", "judge", "deem", "regard",
    "say", "claim", "argue", "maintain", "suggest", "assert", "contend",
    "want", "wish", "hope", "desire", "prefer",
    "like", "love", "enjoy", "admire", "appreciate",
    "dislike", "hate", "detest", "despise",
    "fear", "dread", "regret", "worry", "doubt", "mourn",
    "must", "should", "ought", "need", "have_to",
    "watch", "observe", "notice", "perceive", "recognize",
    "value", "assess", "evaluate", "review", "approve", "disapprove",
    "render", "make", "leave", "drive"
}

# ---------- IO: parse conllu ----------
def parse_conllu_file(path):
    """Return list of sentences; each sentence is a list of token dicts."""
    sentences = []
    sent = []
    with open(path, "r", encoding="utf-8") as fh:
        for line in fh:
            line = line.rstrip("\n")
            if not line:
                if sent:
                    sentences.append(sent)
                    sent = []
                continue
            if line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) < 8:
                continue
            if "-" in parts[0] or "." in parts[0]:
                continue
            try:
                idx = int(parts[0])
            except:
                continue
            token = {
                "id": idx,
                "form": parts[1],
                "lemma": parts[2].lower(),
                "upos": parts[3],
                "head": int(parts[6]) if parts[6].isdigit() else 0,
                "deprel": parts[7],
            }
            sent.append(token)
    if sent:
        sentences.append(sent)
    return sentences

# ---------- classification & example collection ----------
def _append_example(adj_examples, lemma, category, sentence_text):
    """Append sentence_text to adj_examples[lemma][category] up to EXAMPLES_PER_CLASS, avoiding duplicates."""
    lst = adj_examples.setdefault(lemma, {"Affect":[], "Judgement":[], "Appreciation":[], "ambiguous":[]})
    if sentence_text in lst[category]:
        return
    if len(lst[category]) < EXAMPLES_PER_CLASS:
        lst[category].append(sentence_text)

def classify_adjectives_in_sentence(sent, adj_stats, adj_examples):
    by_id = {t["id"]: t for t in sent}
    children = defaultdict(list)
    for t in sent:
        children[t["head"]].append(t)

    sentence_text = " ".join(t["form"] for t in sent)

    for t in sent:
        if t["upos"] != "ADJ":
            continue
        lemma = t["lemma"]
        adj_stats[lemma]["total"] += 1

        subj_candidates = [c for c in children.get(t["id"], []) if c["deprel"].startswith("nsubj")]
        cop_children = [c for c in children.get(t["id"], []) if c["deprel"] == "cop"]
        classified = False

        if subj_candidates:
            subj = subj_candidates[0]
            s_lemma = subj["lemma"]
            s_upos = subj["upos"]
            if s_upos == "PRON" and s_lemma in FIRST_PERSON:
                adj_stats[lemma]["Affect"] += 1
                _append_example(adj_examples, lemma, "Affect", sentence_text)
                classified = True
            elif s_upos == "PRON" and s_lemma in THIRD_PERSON_PRONOUNS:
                adj_stats[lemma]["Judgement"] += 1
                _append_example(adj_examples, lemma, "Judgement", sentence_text)
                classified = True
            elif s_upos == "PROPN":
                adj_stats[lemma]["Judgement"] += 1
                _append_example(adj_examples, lemma, "Judgement", sentence_text)
                classified = True
            elif s_upos == "PRON" and s_lemma in APPRECIATION_PRONOUNS:
                adj_stats[lemma]["Appreciation"] += 1
                _append_example(adj_examples, lemma, "Appreciation", sentence_text)
                classified = True
            elif s_upos == "NOUN":
                adj_stats[lemma]["Appreciation"] += 1
                _append_example(adj_examples, lemma, "Appreciation", sentence_text)
                classified = True

        if not classified and t["deprel"] == "xcomp":
            head = by_id.get(t["head"])
            if head and head["upos"].startswith("V"):
                verb_children = children.get(head["id"], [])
                v_subjs = [c for c in verb_children if c["deprel"].startswith("nsubj")]
                if v_subjs:
                    vsub = v_subjs[0]
                    vl = vsub["lemma"]; vup = vsub["upos"]
                    if vup == "PRON" and vl in FIRST_PERSON:
                        adj_stats[lemma]["Affect"] += 1
                        _append_example(adj_examples, lemma, "Affect", sentence_text)
                        classified = True
                    elif vup == "PRON" and vl in THIRD_PERSON_PRONOUNS:
                        adj_stats[lemma]["Judgement"] += 1
                        _append_example(adj_examples, lemma, "Judgement", sentence_text)
                        classified = True
                    elif vup == "PROPN":
                        adj_stats[lemma]["Judgement"] += 1
                        _append_example(adj_examples, lemma, "Judgement", sentence_text)
                        classified = True
                    elif vup == "NOUN":
                        adj_stats[lemma]["Appreciation"] += 1
                        _append_example(adj_examples, lemma, "Appreciation", sentence_text)
                        classified = True
                else:
                    if head["lemma"] in FEEL_VERBS:
                        adj_stats[lemma]["Affect"] += 1
                        _append_example(adj_examples, lemma, "Affect", sentence_text)
                        classified = True

        if not classified and cop_children:
            if subj_candidates:
                subj = subj_candidates[0]
                sl = subj["lemma"]; sup = subj["upos"]
                if sup == "PRON" and sl in FIRST_PERSON:
                    adj_stats[lemma]["Affect"] += 1
                    _append_example(adj_examples, lemma, "Affect", sentence_text)
                    classified = True
                elif sup == "PRON" and sl in THIRD_PERSON_PRONOUNS:
                    adj_stats[lemma]["Judgement"] += 1
                    _append_example(adj_examples, lemma, "Judgement", sentence_text)
                    classified = True
                elif sup == "PROPN":
                    adj_stats[lemma]["Judgement"] += 1
                    _append_example(adj_examples, lemma, "Judgement", sentence_text)
                    classified = True
                elif sup == "NOUN":
                    adj_stats[lemma]["Appreciation"] += 1
                    _append_example(adj_examples, lemma, "Appreciation", sentence_text)
                    classified = True
                elif sup == "PRON" and sl in APPRECIATION_PRONOUNS:
                    adj_stats[lemma]["Appreciation"] += 1
                    _append_example(adj_examples, lemma, "Appreciation", sentence_text)
                    classified = True

        if not classified:
            adj_stats[lemma]["ambiguous"] += 1
            _append_example(adj_examples, lemma, "ambiguous", sentence_text)

# ---------- aggregation (with thresholding) ----------
def compute_appraisal_with_examples_by_class(root, top_n=None, lang_filter=None, confidence_threshold=0.50):
    all_rows = []
    for lang in sorted(os.listdir(root)):
        if lang_filter and lang != lang_filter:
            continue
        lpath = os.path.join(root, lang)
        if not os.path.isdir(lpath):
            continue
        for Domain in sorted(os.listdir(lpath)):
            dpath = os.path.join(lpath, Domain)
            if not os.path.isdir(dpath):
                continue

            # Capitalize Domain names
            Domain = Domain.capitalize()

            adj_stats = defaultdict(lambda: Counter({"total":0, "Affect":0, "Judgement":0, "Appreciation":0, "ambiguous":0}))
            adj_examples = {}

            for fname in sorted(os.listdir(dpath)):
                if not fname.endswith(".conllu"):
                    continue
                path = os.path.join(dpath, fname)
                sents = parse_conllu_file(path)
                for s in sents:
                    classify_adjectives_in_sentence(s, adj_stats, adj_examples)

            rows = []
            for lemma, cnts in adj_stats.items():
                tot = cnts["total"]
                aff = cnts["Affect"]
                jud = cnts["Judgement"]
                app = cnts["Appreciation"]
                amb = cnts["ambiguous"]
                co_total = aff + jud + app
                if co_total > 0:
                    p_aff = aff / co_total
                    p_jud = jud / co_total
                    p_app = app / co_total
                else:
                    p_aff = p_jud = p_app = 0.0

                probs = {"Affect": p_aff, "Judgement": p_jud, "Appreciation": p_app}
                max_prob = max(probs.values())
                classes_meeting = [k for k, v in probs.items() if v >= confidence_threshold]
                assigned_class = None
                assigned_reason = ""
                is_confident = False

                if len(classes_meeting) == 1:
                    assigned_class = classes_meeting[0]
                    is_confident = True
                    assigned_reason = f"single_class_above_threshold ({confidence_threshold})"
                elif len(classes_meeting) > 1:
                    sorted_by_prob = sorted(probs.items(), key=lambda x: x[1], reverse=True)
                    top_name, top_val = sorted_by_prob[0]
                    second_val = sorted_by_prob[1][1]
                    if top_val > second_val and top_val >= confidence_threshold:
                        assigned_class = top_name
                        is_confident = True
                        assigned_reason = "highest_prob_above_threshold (others below)"
                    else:
                        assigned_class = "ambiguous_tie"
                        assigned_reason = "multiple_classes_above_threshold_or_tied"
                else:
                    if max_prob == 0:
                        assigned_class = "ambiguous_no_evidence"
                        assigned_reason = "no_Affect/Judgement/Appreciation_evidence"
                    else:
                        assigned_class = "ambiguous_low_confidence"
                        assigned_reason = f"max_prob_below_threshold ({max_prob:.3f} < {confidence_threshold})"
                    is_confident = False

                examples = adj_examples.get(lemma, {"Affect":[], "Judgement":[], "Appreciation":[], "ambiguous":[]})
                ex_aff = " || ".join(examples["Affect"])
                ex_jud = " || ".join(examples["Judgement"])
                ex_app = " || ".join(examples["Appreciation"])
                ex_amb = " || ".join(examples["ambiguous"])
                if round(max_prob, 4) > confidence_threshold:
                    rows.append({
                        "language": lang,
                        "Domain": Domain,
                        "adjective": lemma,
                        "total_count": tot,
                        "Affect_count": aff,
                        "Judgement_count": jud,
                        "Appreciation_count": app,
                        "ambiguous_count": amb,
                        "p_Affect": round(p_aff, 4),
                        "p_Judgement": round(p_jud, 4),
                        "p_Appreciation": round(p_app, 4),
                        "assigned_class": assigned_class,
                        "assigned_confidence": round(max_prob, 4),
                        "example_Affect": ex_aff,
                        "example_Judgement": ex_jud,
                        "example_Appreciation": ex_app,
                        "example_ambiguous": ex_amb
                    })

            rows = sorted(rows, key=lambda r: r["total_count"], reverse=True)
            if top_n:
                rows = rows[:top_n]
            all_rows.extend(rows)
    return all_rows

# ---------- run ----------
def main(threshold=CONFIDENCE_THRESHOLD):
    out = compute_appraisal_with_examples_by_class(CONLLU_ROOT, top_n=KEEP_TOP_N, lang_filter="en", confidence_threshold=threshold)
    if not out:
        print("No results (check your CONLLU_ROOT path and folder structure).")
        return

    os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
    with open(OUT_CSV, "w", encoding="utf-8", newline="") as fh:
        writer = csv.DictWriter(fh, fieldnames=list(out[0].keys()))
        writer.writeheader()
        for row in out:
            writer.writerow(row)
    print("Saved:", OUT_CSV)
    print(f"Used confidence threshold = {threshold}")

if __name__ == "__main__":
    main()


Saved: conllu/adjectives_appraisal_with_examples_by_Domain_thresholded.csv
Used confidence threshold = 0.5


In [4]:
!pip install simalign

In [5]:
from simalign import SentenceAligner

# initialize a dummy aligner
aligner = SentenceAligner(model="bert", token_type="bpe")

# inspect the internal attribute that lists available matching methods
print(aligner.matching_methods)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

2025-10-11 08:41:48,177 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


['mwmf', 'inter', 'itermax']


In [6]:
from simalign import SentenceAligner
aligner = SentenceAligner(model="bert", token_type="bpe")
src = ["This", "is", "amazing", "."]
tgt = ["Das", "ist", "erstaunlich", "."]

aligned = aligner.get_word_aligns(src, tgt)
print("aligned (raw):", aligned)
# print available methods and a chosen set
if isinstance(aligned, dict):
    print("methods:", list(aligned.keys()))
    print("itermax:", aligned.get("itermax"))
    print("mwmf:", aligned.get("mwmf"))


2025-10-11 08:41:49,673 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


aligned (raw): {'mwmf': [(0, 0), (1, 1), (2, 2), (3, 3)], 'inter': [(0, 0), (1, 1), (2, 2), (3, 3)], 'itermax': [(0, 0), (1, 1), (2, 2), (3, 3)]}
methods: ['mwmf', 'inter', 'itermax']
itermax: [(0, 0), (1, 1), (2, 2), (3, 3)]
mwmf: [(0, 0), (1, 1), (2, 2), (3, 3)]


In [7]:
import inspect
print(inspect.signature(aligner.get_word_aligns))
print(inspect.getsource(aligner.get_word_aligns))


(src_sent: Union[str, List[str]], trg_sent: Union[str, List[str]]) -> Dict[str, List]
	def get_word_aligns(self, src_sent: Union[str, List[str]], trg_sent: Union[str, List[str]]) -> Dict[str, List]:
		if isinstance(src_sent, str):
			src_sent = src_sent.split()
		if isinstance(trg_sent, str):
			trg_sent = trg_sent.split()
		l1_tokens = [self.embed_loader.tokenizer.tokenize(word) for word in src_sent]
		l2_tokens = [self.embed_loader.tokenizer.tokenize(word) for word in trg_sent]
		bpe_lists = [[bpe for w in sent for bpe in w] for sent in [l1_tokens, l2_tokens]]

		if self.token_type == "bpe":
			l1_b2w_map = []
			for i, wlist in enumerate(l1_tokens):
				l1_b2w_map += [i for x in wlist]
			l2_b2w_map = []
			for i, wlist in enumerate(l2_tokens):
				l2_b2w_map += [i for x in wlist]

		vectors = self.embed_loader.get_embed_list([src_sent, trg_sent]).cpu().detach().numpy()
		vectors = [vectors[i, :len(bpe_lists[i])] for i in [0, 1]]

		if self.token_type == "word":
			vectors = self.av

In [8]:
from tqdm import tqdm

In [9]:
#!/usr/bin/env python3
"""
Scan a directory of CoNLL-U files and collect EVERY ADJ token instance into one CSV.

Place this file next to your existing project and adjust the CONFIG section below.

Output: a single CSV (OUT_INSTANCES_CSV) with one row per ADJ token instance.
Columns: file, Domain, sent_index, token_id, token_form, token_lemma, upos, deprel, head, sense, sentence

The `classify_single_adj_instance` function is unchanged from your original pipeline
and will be used to guess Affect/Judgement/Appreciation/ambiguous.

Run: python collect_all_adjectives.py
"""

import os
import csv
from collections import defaultdict, Counter

# ---------- CONFIG (edit these) ----------
CONLLU_ROOT = "conllu/conllu_file"   # base folder you described
LANG = "en"
OUT_INSTANCES_CSV = "outputs/all_adjective_instances_en.csv"
# If you only want a sample per lemma, set SAMPLE_PER_LEMMA to an int > 0.
# Otherwise set to 0 to keep every instance (can be large).
SAMPLE_PER_LEMMA = 0
# ---------- end CONFIG ----------

FIRST_PERSON = {
    "i", "me", "we", "us",
    "my", "our",
    "mine", "ours",
    "myself", "ourselves"
}
SECOND_PERSON = {"you", "your", "yours", "yourself", "yourselves"}
THIRD_PERSON_PRONOUNS = {"he", "she", "it", "they", "him", "her", "them", "his", "hers", "theirs"}
APPRECIATION_PRONOUNS = {
    "it", "there", "here", "this", "that", "these", "those",
    "something", "anything", "nothing", "everything",
    "someone", "anyone", "no one", "everyone",
    "somebody", "anybody", "nobody", "everybody",
    "all", "each", "both", "neither", "none", "one",
    "what", "whatever", "which", "whichever",
    "this one", "that one", "such", "so"
}
FEEL_VERBS = {
    "be", "become", "get", "remain", "stay", "keep", "prove", "turn",
    "seem", "appear", "sound", "look", "smell", "taste", "feel",
    "think", "consider", "believe", "find", "judge", "deem", "regard",
    "say", "claim", "argue", "maintain", "suggest", "assert", "contend",
    "want", "wish", "hope", "desire", "prefer",
    "like", "love", "enjoy", "admire", "appreciate",
    "dislike", "hate", "detest", "despise",
    "fear", "dread", "regret", "worry", "doubt", "mourn",
    "must", "should", "ought", "need", "have_to",
    "watch", "observe", "notice", "perceive", "recognize",
    "value", "assess", "evaluate", "review", "approve", "disapprove",
    "render", "make", "leave", "drive"
}

# --- helper: read conllu into list of sentences (as list of token dicts)
def parse_conllu_file(path):
    sentences = []
    sent = []
    with open(path, "r", encoding="utf-8") as fh:
        for line in fh:
            line = line.rstrip("\n")
            if not line:
                if sent:
                    sentences.append(sent)
                    sent = []
                continue
            if line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) < 8:
                continue
            if "-" in parts[0] or "." in parts[0]:
                continue
            try:
                idx = int(parts[0])
            except:
                continue
            token = {
                "id": idx,
                "form": parts[1],
                "lemma": parts[2].lower(),
                "upos": parts[3],
                "head": int(parts[6]) if parts[6].isdigit() else 0,
                "deprel": parts[7],
            }
            sent.append(token)
    if sent:
        sentences.append(sent)
    return sentences

# --- classification logic adapted from your pipeline to return per-token labels
def classify_single_adj_instance(token, sent):
    """
    Returns one of: "Affect", "Judgement", "Appreciation", "ambiguous"
    token: the ADJ token dict
    sent: list of token dicts
    """
    by_id = {t["id"]: t for t in sent}
    children = defaultdict(list)
    for t in sent:
        children[t["head"]].append(t)

    subj_candidates = [c for c in children.get(token["id"], []) if c["deprel"].startswith("nsubj")]
    cop_children = [c for c in children.get(token["id"], []) if c["deprel"] == "cop"]

    if subj_candidates:
        subj = subj_candidates[0]
        s_lemma = subj["lemma"]; s_upos = subj["upos"]
        if s_upos == "PRON" and s_lemma in FIRST_PERSON:
            return "Affect"
        if s_upos == "PRON" and s_lemma in THIRD_PERSON_PRONOUNS:
            return "Judgement"
        if s_upos == "PROPN":
            return "Judgement"
        if s_upos == "PRON" and s_lemma in APPRECIATION_PRONOUNS:
            return "Appreciation"
        if s_upos == "NOUN":
            return "Appreciation"

    # xcomp case
    if token["deprel"] == "xcomp":
        head = by_id.get(token["head"])
        if head and head["upos"].startswith("V"):
            verb_children = children.get(head["id"], [])
            v_subjs = [c for c in verb_children if c["deprel"].startswith("nsubj")]
            if v_subjs:
                vsub = v_subjs[0]; vl = vsub["lemma"]; vup = vsub["upos"]
                if vup == "PRON" and vl in FIRST_PERSON:
                    return "Affect"
                if vup == "PRON" and vl in THIRD_PERSON_PRONOUNS:
                    return "Judgement"
                if vup == "PROPN":
                    return "Judgement"
                if vup == "NOUN":
                    return "Appreciation"
            else:
                if head["lemma"] in FEEL_VERBS:
                    return "Affect"

    if cop_children:
        if subj_candidates:
            subj = subj_candidates[0]; sl = subj["lemma"]; sup = subj["upos"]
            if sup == "PRON" and sl in FIRST_PERSON:
                return "Affect"
            if sup == "PRON" and sl in THIRD_PERSON_PRONOUNS:
                return "Judgement"
            if sup == "PROPN":
                return "Judgement"
            if sup == "NOUN":
                return "Appreciation"
            if sup == "PRON" and sl in APPRECIATION_PRONOUNS:
                return "Appreciation"

    return "ambiguous"


def collect_all_adjectives(conllu_root, lang="en"):
    """Return list of dict rows for every ADJ token found under conllu_root/lang/*/*.conllu"""
    rows = []
    langpath = os.path.join(conllu_root, lang)
    if not os.path.isdir(langpath):
        raise FileNotFoundError(langpath)
    for Domain in sorted(os.listdir(langpath)):
        dpath = os.path.join(langpath, Domain)
        if not os.path.isdir(dpath):
            continue
        for fname in sorted(os.listdir(dpath)):
            if not fname.endswith(".conllu"):
                continue
            fpath = os.path.join(dpath, fname)
            sents = parse_conllu_file(fpath)
            for sent_idx, sent in enumerate(sents):
                sent_text = " ".join(t["form"] for t in sent)
                for t in sent:
                    if t["upos"] == "ADJ":
                        sense = classify_single_adj_instance(t, sent)
                        rows.append({
                            "file": fname,
                            "Domain": Domain,
                            "sent_index": sent_idx,
                            "token_id": t["id"],
                            "token_form": t["form"],
                            "token_lemma": t["lemma"],
                            "upos": t["upos"],
                            "deprel": t["deprel"],
                            "head": t["head"],
                            "sense": sense,
                            "sentence": sent_text
                        })
    return rows


def main():
    os.makedirs(os.path.dirname(OUT_INSTANCES_CSV), exist_ok=True)
    rows = collect_all_adjectives(CONLLU_ROOT, lang=LANG)
    if not rows:
        print("No ADJ instances found.")
        return

    # === NEW: filter out ambiguous instances (keep only non-ambiguous) ===
    # User requested: "keep only non ambigious only"
    # This will drop any row where sense == "ambiguous".
    rows = [r for r in rows if r.get("sense") != "ambiguous"]
    if not rows:
        print("No non-ambiguous ADJ instances found after filtering.")
        return

    # If sampling requested, keep up to SAMPLE_PER_LEMMA examples per lemma
    if SAMPLE_PER_LEMMA and SAMPLE_PER_LEMMA > 0:
        samples = []
        by_lemma = defaultdict(list)
        for r in rows:
            by_lemma[r["token_lemma"]].append(r)
        for lemma, lst in by_lemma.items():
            samples.extend(lst[:SAMPLE_PER_LEMMA])
        rows_to_write = samples
    else:
        rows_to_write = rows

    # write CSV
    keys = list(rows_to_write[0].keys())
    with open(OUT_INSTANCES_CSV, "w", encoding="utf-8", newline="") as fh:
        w = csv.DictWriter(fh, fieldnames=keys)
        w.writeheader()
        for r in rows_to_write:
            w.writerow(r)
    print("Saved instances:", OUT_INSTANCES_CSV)

    # quick counts (on the full set before sampling)
    c = Counter(r["sense"] for r in rows)
    tot = sum(c.values())
    print("Total non-ambiguous ADJ instances:", tot)
    print("Sense breakdown:", dict(c))

    # top lemmas
    lemma_counts = Counter(r["token_lemma"] for r in rows)
    print("Top 30 lemmas:")
    for lemma, cnt in lemma_counts.most_common(30):
        print(f"  {lemma}: {cnt}")

if __name__ == "__main__":
    main()


Saved instances: outputs/all_adjective_instances_en.csv
Total non-ambiguous ADJ instances: 4459
Sense breakdown: {'Appreciation': 2143, 'Affect': 1165, 'Judgement': 1151}
Top 30 lemmas:
  good: 223
  able: 149
  important: 85
  happy: 78
  right: 73
  wrong: 73
  old: 66
  true: 65
  sure: 63
  bad: 61
  easy: 54
  different: 54
  interested: 47
  hard: 43
  interesting: 40
  great: 39
  likely: 39
  big: 36
  same: 35
  close: 34
  amazing: 33
  high: 32
  excited: 30
  possible: 30
  simple: 29
  safe: 26
  worth: 25
  sorry: 25
  small: 25
  available: 25


In [10]:
def load_sentences_from_file(path): return parse_conllu_file(path)

In [11]:
#!/usr/bin/env python3
# extract_all_adjectives_translations.py
import os
import csv
import json
from collections import defaultdict
from tqdm import tqdm

from simalign import SentenceAligner

# ---------- CONFIG ----------
CONLLU_ROOT = "conllu/conllu_file"
LANG_EN = "en"
LANG_DE = "de"
INSTANCES_CSV = "outputs/all_adjective_instances_en.csv"   # <-- input with all adjectives
OUT_TRANSLATIONS_CSV = "outputs/all_adjectives_en_de_translations.csv"
ALIGN_MODE = "itermax"  # not directly passed to get_word_aligns; we will pick best returned key
# ----------------------------

def parse_conllu_file(path):
    sentences = []
    sent = []
    with open(path, "r", encoding="utf-8") as fh:
        for line in fh:
            line = line.rstrip("\n")
            if not line:
                if sent:
                    sentences.append(sent)
                    sent = []
                continue
            if line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) < 8:
                continue
            if "-" in parts[0] or "." in parts[0]:
                continue
            try:
                idx = int(parts[0])
            except:
                continue
            token = {
                "id": idx,
                "form": parts[1],
                "lemma": parts[2].lower(),
                "upos": parts[3],
                "head": int(parts[6]) if parts[6].isdigit() else 0,
                "deprel": parts[7],
            }
            sent.append(token)
    if sent:
        sentences.append(sent)
    return sentences

def load_sentences_from_file(path):
    return parse_conllu_file(path)

def main():
    os.makedirs(os.path.dirname(OUT_TRANSLATIONS_CSV), exist_ok=True)

    # Load instances CSV (expected columns include at least:
    # file, Domain, sent_index, token_id, token_form, token_lemma, sense, sentence)
    instances = []
    with open(INSTANCES_CSV, "r", encoding="utf-8") as fh:
        rdr = csv.DictReader(fh)
        for r in rdr:
            # try to coerce numeric fields, but be robust if missing
            try:
                r["sent_index"] = int(r.get("sent_index", -1))
            except:
                r["sent_index"] = -1
            try:
                r["token_id"] = int(r.get("token_id", -1))
            except:
                r["token_id"] = -1
            # normalize some expected field names if necessary
            # ensure lower-case lemma column exists as token_lemma
            if "token_lemma" not in r and "lemma" in r:
                r["token_lemma"] = r.get("lemma", "")
            # ensure token_form exists
            if "token_form" not in r and "form" in r:
                r["token_form"] = r.get("form", "")
            instances.append(r)

    # index instances by file+sent_index for fast lookup
    instances_by_file_sent = defaultdict(list)
    for inst in instances:
        # Normalize filename key by removing _en/_de suffix if present (as in your earlier logic)
        fname_norm = inst.get("file", "").replace("_en", "").replace("_de", "")
        key = (inst.get("Domain", ""), fname_norm)
        instances_by_file_sent[(key, inst["sent_index"])].append(inst)

    # prepare aligner
    aligner = SentenceAligner(model="bert", token_type="bpe")

    out_rows = []
    en_root = os.path.join(CONLLU_ROOT, LANG_EN)

    # iterate domains and files in EN folder
    for Domain in tqdm(sorted(os.listdir(en_root)), desc="Processing Domains"):
        dpath = os.path.join(en_root, Domain)
        if not os.path.isdir(dpath):
            continue
        for fname in tqdm(sorted(os.listdir(dpath)), desc=f"Files in {Domain}", leave=False):
            if not fname.endswith(".conllu"):
                continue
            en_fpath = os.path.join(dpath, fname)
            key = fname.replace("_en","").replace("_de","")

            # find corresponding DE file (try _de variant, then fallback to same name without suffix)
            de_candidate = fname.replace("_en","_de")
            de_fpath = os.path.join(CONLLU_ROOT, LANG_DE, Domain, de_candidate)
            if not os.path.exists(de_fpath):
                de_fpath = os.path.join(CONLLU_ROOT, LANG_DE, Domain, fname.replace("_en",""))
                if not os.path.exists(de_fpath):
                    # no matching DE file, skip but keep user informed
                    # Instances might still expect a match; we skip
                    # (You can change to continue or to output empty German fields)
                    # print is kept intentionally for debug
                    print("Skipping (no matching DE file):", en_fpath)
                    continue

            # load sentences
            en_sents = load_sentences_from_file(en_fpath)
            de_sents = load_sentences_from_file(de_fpath)
            if len(en_sents) != len(de_sents):
                print(f"Warning: sentence counts differ for {fname}: EN={len(en_sents)} DE={len(de_sents)} -- proceeding with min length")
            n = min(len(en_sents), len(de_sents))

            # iterate sentence indices that have instances
            for sent_idx in range(n):
                key_tuple = ((Domain, key), sent_idx)
                insts = instances_by_file_sent.get(key_tuple)
                if not insts:
                    continue

                en_sent = en_sents[sent_idx]
                de_sent = de_sents[sent_idx]
                en_tokens = [t["form"] for t in en_sent]
                de_tokens = [t["form"] for t in de_sent]

                # compute alignment for this sentence pair
                try:
                    aligned = aligner.get_word_aligns(en_tokens, de_tokens)
                except Exception as e:
                    print("Alignment error for file", fname, "sent", sent_idx, "->", str(e))
                    aligned = {}

                # Extract alignment pairs robustly:
                pairs = []
                # aligned may be a dict with keys like 'itermax', 'mwmf', 'ma', 'za'
                if isinstance(aligned, dict):
                    # try preferred mode first
                    for k in (ALIGN_MODE, "mwmf", "itermax", "ma", "za"):
                        v = aligned.get(k)
                        if v:
                            pairs = v
                            break
                    # fallback to generic 'alignments' key or list of tuples
                    if not pairs:
                        if "alignments" in aligned and aligned["alignments"]:
                            pairs = aligned["alignments"]
                elif isinstance(aligned, list):
                    pairs = aligned

                # build mapping from en_index0 -> [de_index0, ...]
                en2de = defaultdict(list)
                for a in pairs:
                    if isinstance(a, str) and "-" in a:
                        try:
                            i_s, j_s = a.split("-")
                            i = int(i_s); j = int(j_s)
                        except:
                            continue
                    elif isinstance(a, (list, tuple)) and len(a) == 2:
                        try:
                            i = int(a[0]); j = int(a[1])
                        except:
                            continue
                    else:
                        continue
                    en2de[i].append(j)

                # For each instance in this sentence, extract aligned German spans and write row
                for inst in insts:
                    token_id = inst.get("token_id", -1)
                    en_index0 = token_id - 1
                    aligned_de_indices = en2de.get(en_index0, [])

                    if not aligned_de_indices:
                        aligned_forms = ""
                        aligned_lemmas = ""
                    else:
                        forms = [de_sent[j]["form"] for j in sorted(aligned_de_indices) if 0 <= j < len(de_sent)]
                        lemmas = [de_sent[j]["lemma"] for j in sorted(aligned_de_indices) if 0 <= j < len(de_sent)]
                        aligned_forms = " ".join(forms)
                        aligned_lemmas = " ".join(lemmas)

                    # build output row: include user's requested columns plus German info
                    out_row = {
                        "file": fname,
                        "Domain": Domain,
                        "sent_index": sent_idx,
                        "token_id": token_id,
                        "token_form": inst.get("token_form", ""),
                        "token_lemma": inst.get("token_lemma", ""),
                        "sense": inst.get("sense", ""),
                        "sentence": inst.get("sentence", " ".join(t["form"] for t in en_sent)),
                        # additional useful columns
                        "de_aligned_forms": aligned_forms,
                        "de_aligned_lemmas": aligned_lemmas,
                        "de_sentence": " ".join(t["form"] for t in de_sent),
                    }
                    out_rows.append(out_row)
                    print(out_row)

    # write results to CSV and JSON
    keys = ["file","Domain","sent_index","token_id","token_form","token_lemma","sense","sentence",
            "de_aligned_forms","de_aligned_lemmas","de_sentence"]
    # ensure keys cover existing rows
    if out_rows:
        # derive any extra keys present and append (keeps deterministic order)
        extra = [k for k in out_rows[0].keys() if k not in keys]
        keys.extend(extra)

    with open(OUT_TRANSLATIONS_CSV, "w", encoding="utf-8", newline="") as fh:
        if out_rows:
            w = csv.DictWriter(fh, fieldnames=keys)
            w.writeheader()
            for r in out_rows:
                w.writerow(r)
    print("Saved translation extractions:", OUT_TRANSLATIONS_CSV)

    # also write JSON for easier inspection if desired
    json_path = OUT_TRANSLATIONS_CSV.replace(".csv", ".json")
    with open(json_path, "w", encoding="utf-8") as fh:
        json.dump(out_rows, fh, ensure_ascii=False, indent=2)
    print("Also wrote JSON:", json_path)

if __name__ == "__main__":
    main()

2025-10-11 08:51:13,636 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
Files in Art:   0%|          | 0/18 [00:00<?, ?it/s]

{'file': 'Art10_en.conllu', 'Domain': 'Art', 'sent_index': 7, 'token_id': 10, 'token_form': 'safer', 'token_lemma': 'safe', 'sense': 'Appreciation', 'sentence': 'I had come to the conclusion that homelessness was safer for me than being at home .', 'de_aligned_forms': 'sicherer', 'de_aligned_lemmas': 'sicher', 'de_sentence': 'Ich beschloss , dass kein Zuhause sicherer war als meins .'}
{'file': 'Art10_en.conllu', 'Domain': 'Art', 'sent_index': 9, 'token_id': 14, 'token_form': 'uncared', 'token_lemma': 'uncared', 'sense': 'Affect', 'sentence': "During my homelessness , I joined Atlanta 's 3,300 homeless youth in feeling uncared for , left out and invisible each night .", 'de_aligned_forms': 'unerwünscht überflüssig unsichtbar', 'de_aligned_lemmas': 'unerwünscht überflüssig unsichtbar', 'de_sentence': 'Damals gehörte ich zu Atlantas 3 300 obdachlosen Jugendlichen , die sich unerwünscht fühlten , jede Nacht überflüssig und unsichtbar .'}
{'file': 'Art10_en.conllu', 'Domain': 'Art', 'sent_


Files in Art:  11%|█         | 2/18 [00:04<00:39,  2.44s/it]

{'file': 'Art10_en.conllu', 'Domain': 'Art', 'sent_index': 35, 'token_id': 30, 'token_form': 'heavy', 'token_lemma': 'heavy', 'sense': 'Appreciation', 'sentence': 'We build with Liz , who has been on the streets most of her teenage years but turns to music to return to herself when her traumas feel too heavy for her young shoulders .', 'de_aligned_forms': 'schwer', 'de_aligned_lemmas': 'schwer', 'de_sentence': 'Wir bauen auf mit Liz , die bereits einen Großteil ihrer Jugend auf der Straße gelebt hat , aber sich der Musik zuwendet , um zu sich zu finden , wenn ihr Trauma zu schwer für ihre jungen Schultern wird .'}
{'file': 'Art11_en.conllu', 'Domain': 'Art', 'sent_index': 4, 'token_id': 27, 'token_form': 'addicted', 'token_lemma': 'addicted', 'sense': 'Appreciation', 'sentence': 'When I was a kid , I loved to draw , and the most talented artist I knew was my mother , but my mother was addicted to heroin .', 'de_aligned_forms': 'Künstlerin', 'de_aligned_lemmas': 'künstlerin', 'de_senten


Files in Art:  17%|█▋        | 3/18 [00:40<04:03, 16.21s/it]

...


Files in Tech:  96%|█████████▌| 25/26 [03:27<00:06,  6.95s/it]

{'file': 'Tech8_en.conllu', 'Domain': 'Tech', 'sent_index': 85, 'token_id': 21, 'token_form': 'safe', 'token_lemma': 'safe', 'sense': 'Affect', 'sentence': "It 's just that for those of us dealing with scale , for those of us tasked with keeping people safe , we have to assume the worst will happen , because for us , a one - in - a - million chance is pretty good odds .", 'de_aligned_forms': 'schützen', 'de_aligned_lemmas': 'schützen', 'de_sentence': 'Nur wir , die mit dieser Größenordnung arbeiten , die den Auftrag haben , Menschen zu schützen , müssen davon ausgehen , dass das Schlimmste passiert . Weil für uns eine Wahrscheinlichkeit von 1:1 000 000 eine ziemlich realistische Wette ist .'}
{'file': 'Tech9_en.conllu', 'Domain': 'Tech', 'sent_index': 7, 'token_id': 10, 'token_form': 'missing', 'token_lemma': 'missing', 'sense': 'Appreciation', 'sentence': 'After my neurosurgery , part of my brain was missing , and I had to deal with that .', 'de_aligned_forms': 'fehlte', 'de_aligned_l


Processing Domains: 100%|██████████| 11/11 [37:07<00:00, 202.54s/it]

{'file': 'Tech9_en.conllu', 'Domain': 'Tech', 'sent_index': 75, 'token_id': 6, 'token_form': 'big', 'token_lemma': 'big', 'sense': 'Judgement', 'sentence': 'You think the Internet was big .', 'de_aligned_forms': 'groß', 'de_aligned_lemmas': 'groß', 'de_sentence': 'Sie denken , das Internet sei groß .'}
Saved translation extractions: outputs/all_adjectives_en_de_translations.csv


Also wrote JSON: outputs/all_adjectives_en_de_translations.json


In [16]:
#!/usr/bin/env python3
"""
aggregate_adjectives_by_sense_with_counts.py

Input (default):
  outputs/all_adjectives_en_de_translations.csv

Output (default):
  outputs/all_adjectives_en_de_aggregated_with_counts.csv

Each row: en_form, en_sense, de_aligned_forms, de_aligned_counts,
          example_en_sentence, example_de_sentence,
          n_instances, total_en_form_instances, pct_of_form
"""
import os
import csv
import pandas as pd
from collections import Counter

INPUT_CSV = "outputs/all_adjectives_en_de_translations.csv"
OUTPUT_CSV = "outputs/all_adjectives_en_de_aggregated_with_counts.csv"

def find_col(df, candidates, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    return default

def normalize_sense(s):
    if not s or pd.isna(s):
        return "(NULL)"
    s0 = str(s).strip().lower()
    # unify common variants
    if s0 in ("judgment", "judgement"):
        return "Judgement"
    if s0 == "affect":
        return "Affect"
    if s0 == "appreciation":
        return "Appreciation"
    # fallback: title case
    return s0.capitalize()

def choose_example(group, en_col, de_col):
    # prefer a row where de_col is non-empty, else any row
    for _, r in group.iterrows():
        if r.get(de_col) and str(r.get(de_col)).strip():
            return r.get(en_col, ""), r.get(de_col, "")
    # fallback to first row
    r = group.iloc[0]
    return r.get(en_col, ""), r.get(de_col, "")

def main(input_csv=INPUT_CSV, output_csv=OUTPUT_CSV):
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    df = pd.read_csv(input_csv, dtype=str).fillna("")
    # tolerant column lookup
    token_col = find_col(df, ["token_form","en_form","form","token"], default="token_form")
    sense_col = find_col(df, ["sense","en_sense"], default="sense")
    de_forms_col = find_col(df, ["de_aligned_forms","de_aligned_form","de_forms"], default="de_aligned_forms")
    de_lem_col = find_col(df, ["de_aligned_lemmas","de_aligned_lemma","de_lemmas"], default="de_aligned_lemmas")
    en_sent_col = find_col(df, ["sentence","en_sentence","en_sent"], default="sentence")
    de_sent_col = find_col(df, ["de_sentence","de_sent"], default="de_sentence")

    # construct a unified de_label column (prefer forms, fallback to lemmas)
    def best_de_label(row):
        forms = str(row.get(de_forms_col, "")).strip()
        if forms:
            return forms
        lem = str(row.get(de_lem_col, "")).strip()
        if lem:
            return lem
        return "(NULL)"

    df["de_label"] = df.apply(best_de_label, axis=1)
    # normalize en_form and sense for grouping (keep original en_form as-is in output)
    df["en_form_norm"] = df[token_col].fillna("").astype(str).str.strip()
    df["sense_norm"] = df[sense_col].fillna("").astype(str).str.strip().str.lower()

    results = []
    # Precompute total counts per en_form (across senses)
    total_per_form = df.groupby("en_form_norm").size().to_dict()

    # group by en_form and sense_norm
    grouped = df.groupby(["en_form_norm", "sense_norm"])

    for (en_form, sense_norm), group in grouped:
        if not en_form:
            continue
        display_sense = normalize_sense(sense_norm)
        # count distinct de_label spans (treat each cell string as one span)
        labels = list(group["de_label"].astype(str).map(str.strip))
        counts = Counter(labels)
        # sort by descending freq then alphabetic
        sorted_labels = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
        de_spans = ", ".join([lbl for lbl, cnt in sorted_labels])
        de_counts = ", ".join([f"{lbl}:{cnt}" for lbl, cnt in sorted_labels])

        # choose example pair: prefer rows where de_label not (NULL)
        example_en, example_de = choose_example(group, en_sent_col, de_sent_col)
        example_en = example_en if example_en is not None else ""
        example_de = example_de if example_de is not None else ""

        n_instances = len(group)
        total_en_form_instances = int(total_per_form.get(en_form, 0))
        pct_of_form = round(n_instances / total_en_form_instances, 4) if total_en_form_instances > 0 else 0.0

        results.append({
            "en_form": en_form,
            "en_sense": display_sense,
            "de_aligned_forms": de_spans,
            "de_aligned_counts": de_counts,
            "example_en_sentence": example_en,
            "example_de_sentence": example_de,
            "n_instances": n_instances,
            "total_en_form_instances": total_en_form_instances,
            "pct_of_form": pct_of_form
        })

    out_df = pd.DataFrame(results, columns=[
        "en_form","en_sense","de_aligned_forms","de_aligned_counts",
        "example_en_sentence","example_de_sentence",
        "n_instances","total_en_form_instances","pct_of_form"
    ])
    out_df.to_csv(output_csv, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)
    print(f"Wrote {len(out_df)} rows to {output_csv}")

    # Print example: if 'amazing' exists, show its sense counts (helpful quick-check)
    target = "amazing"
    if target in total_per_form:
        print(f"Quick check — total occurrences of '{target}': {total_per_form[target]}")
        tdf = out_df[out_df["en_form"].str.lower() == target]
        if not tdf.empty:
            print("Counts per sense for 'amazing':")
            print(tdf[["en_sense","n_instances","pct_of_form"]].to_string(index=False))
        else:
            print("No rows for 'amazing' in aggregated output (maybe casing differs).")

if __name__ == "__main__":
    main()


Wrote 1583 rows to outputs/all_adjectives_en_de_aggregated_with_counts.csv
Quick check — total occurrences of 'amazing': 33
Counts per sense for 'amazing':
    en_sense  n_instances  pct_of_form
      Affect            1       0.0303
Appreciation           24       0.7273
   Judgement            8       0.2424


In [17]:
#!/usr/bin/env python3
"""
aggregate_adjectives_by_sense_with_counts_topk.py

Input (default):
  outputs/all_adjectives_en_de_translations.csv

Output (default):
  outputs/all_adjectives_en_de_aggregated_with_counts_topk.csv

Each row: en_form, en_sense, de_aligned_forms, de_aligned_counts,
          example_en_sentence, example_de_sentence,
          n_instances, total_en_form_instances, pct_of_form

Only includes the top-K English adjectives by total_en_form_instances (default K=20).
"""
import os
import csv
import pandas as pd
from collections import Counter

# ---------- CONFIG ----------
INPUT_CSV = "outputs/all_adjectives_en_de_translations.csv"
OUTPUT_CSV = "outputs/all_adjectives_en_de_aggregated_with_counts_topk.csv"
TOP_K = 20
# ----------------------------

def find_col(df, candidates, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    return default

def normalize_sense(s):
    if not s or pd.isna(s):
        return "(NULL)"
    s0 = str(s).strip().lower()
    # unify common variants
    if s0 in ("judgment", "judgement"):
        return "Judgement"
    if s0 == "affect":
        return "Affect"
    if s0 == "appreciation":
        return "Appreciation"
    # fallback: title case
    return s0.capitalize()

def choose_example(group, en_col, de_col):
    # prefer a row where de_col is non-empty, else any row
    for _, r in group.iterrows():
        if r.get(de_col) and str(r.get(de_col)).strip():
            return r.get(en_col, ""), r.get(de_col, "")
    # fallback to first row
    r = group.iloc[0]
    return r.get(en_col, ""), r.get(de_col, "")

def main(input_csv=INPUT_CSV, output_csv=OUTPUT_CSV, top_k=TOP_K):
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    df = pd.read_csv(input_csv, dtype=str).fillna("")
    # tolerant column lookup
    token_col = find_col(df, ["token_form","en_form","form","token"], default="token_form")
    sense_col = find_col(df, ["sense","en_sense"], default="sense")
    de_forms_col = find_col(df, ["de_aligned_forms","de_aligned_form","de_forms"], default="de_aligned_forms")
    de_lem_col = find_col(df, ["de_aligned_lemmas","de_aligned_lemma","de_lemmas"], default="de_aligned_lemmas")
    en_sent_col = find_col(df, ["sentence","en_sentence","en_sent"], default="sentence")
    de_sent_col = find_col(df, ["de_sentence","de_sent"], default="de_sentence")

    # construct a unified de_label column (prefer forms, fallback to lemmas)
    def best_de_label(row):
        forms = str(row.get(de_forms_col, "")).strip()
        if forms:
            return forms
        lem = str(row.get(de_lem_col, "")).strip()
        if lem:
            return lem
        return "(NULL)"

    df["de_label"] = df.apply(best_de_label, axis=1)
    # normalize en_form and sense for grouping (keep original en_form as-is in output)
    df["en_form_norm"] = df[token_col].fillna("").astype(str).str.strip()
    df["sense_norm"] = df[sense_col].fillna("").astype(str).str.strip().str.lower()

    results = []
    # Precompute total counts per en_form (across senses)
    total_per_form = df.groupby("en_form_norm").size().to_dict()

    # select top-K forms by total occurrences
    sorted_forms = sorted(total_per_form.items(), key=lambda x: (-x[1], x[0]))
    top_forms = [f for f, cnt in sorted_forms[:top_k]]
    top_forms_set = set(top_forms)
    print(f"Selected TOP_K={top_k} forms (by total occurrences). Writing only these {len(top_forms)} forms to output.")
    for f, cnt in sorted_forms[:top_k]:
        print(f"  {f}: {cnt}")

    # group by en_form and sense_norm
    grouped = df.groupby(["en_form_norm", "sense_norm"])

    for (en_form, sense_norm), group in grouped:
        if not en_form:
            continue
        # only include top-K forms
        if en_form not in top_forms_set:
            continue

        display_sense = normalize_sense(sense_norm)
        # count distinct de_label spans (treat each cell string as one span)
        labels = list(group["de_label"].astype(str).map(str.strip))
        counts = Counter(labels)
        # sort by descending freq then alphabetic
        sorted_labels = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
        de_spans = ", ".join([lbl for lbl, cnt in sorted_labels])
        de_counts = ", ".join([f"{lbl}:{cnt}" for lbl, cnt in sorted_labels])

        # choose example pair: prefer rows where de_label not (NULL)
        example_en, example_de = choose_example(group, en_sent_col, de_sent_col)
        example_en = example_en if example_en is not None else ""
        example_de = example_de if example_de is not None else ""

        n_instances = len(group)
        total_en_form_instances = int(total_per_form.get(en_form, 0))
        pct_of_form = round(n_instances / total_en_form_instances, 4) if total_en_form_instances > 0 else 0.0

        results.append({
            "en_form": en_form,
            "en_sense": display_sense,
            "de_aligned_forms": de_spans,
            "de_aligned_counts": de_counts,
            "example_en_sentence": example_en,
            "example_de_sentence": example_de,
            "n_instances": n_instances,
            "total_en_form_instances": total_en_form_instances,
            "pct_of_form": pct_of_form
        })

    out_df = pd.DataFrame(results, columns=[
        "en_form","en_sense","de_aligned_forms","de_aligned_counts",
        "example_en_sentence","example_de_sentence",
        "n_instances","total_en_form_instances","pct_of_form"
    ])
    out_df.to_csv(output_csv, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)
    print(f"Wrote {len(out_df)} rows to {output_csv}")

if __name__ == "__main__":
    main()


Selected TOP_K=20 forms (by total occurrences). Writing only these 20 forms to output.
  able: 149
  good: 112
  better: 104
  important: 85
  right: 73
  wrong: 73
  true: 65
  sure: 63
  happy: 62
  different: 54
  old: 52
  interested: 47
  interesting: 40
  bad: 39
  likely: 39
  great: 36
  easy: 35
  same: 35
  amazing: 33
  hard: 33
Wrote 59 rows to outputs/all_adjectives_en_de_aggregated_with_counts_topk.csv


# Domain wise


In [18]:
#!/usr/bin/env python3
"""
aggregate_adjectives_by_sense_by_domain.py

Input (default):
  outputs/all_adjectives_en_de_translations.csv

Output (default):
  outputs/all_adjectives_en_de_aggregated_by_domain.csv

Each row: en_form, domain, en_sense, de_aligned_forms, de_aligned_counts,
          example_en_sentence, example_de_sentence,
          n_instances, total_en_form_instances_in_domain, pct_of_form_in_domain

For each (en_form, domain) this writes exactly three rows for senses:
  Affect, Judgement, Appreciation
"""
import os
import csv
import pandas as pd
from collections import Counter

INPUT_CSV = "outputs/all_adjectives_en_de_translations.csv"
OUTPUT_CSV = "outputs/all_adjectives_en_de_aggregated_by_domain.csv"

CANONICAL_SENSES = ["Affect", "Judgement", "Appreciation"]

def find_col(df, candidates, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    return default

def normalize_sense(s):
    if not s or pd.isna(s):
        return "(NULL)"
    s0 = str(s).strip().lower()
    # unify common variants
    if s0 in ("judgment", "judgement"):
        return "Judgement"
    if s0 == "affect":
        return "Affect"
    if s0 == "appreciation":
        return "Appreciation"
    # fallback: title case
    return s0.capitalize()

def choose_example(group, en_col, de_col):
    # prefer a row where de_col is non-empty, else any row
    for _, r in group.iterrows():
        if r.get(de_col) and str(r.get(de_col)).strip():
            return r.get(en_col, ""), r.get(de_col, "")
    if len(group) > 0:
        r = group.iloc[0]
        return r.get(en_col, ""), r.get(de_col, "")
    return "", ""

def main(input_csv=INPUT_CSV, output_csv=OUTPUT_CSV):
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    df = pd.read_csv(input_csv, dtype=str).fillna("")

    # tolerant column lookup
    token_col = find_col(df, ["token_form","en_form","form","token"], default="token_form")
    sense_col = find_col(df, ["sense","en_sense"], default="sense")
    de_forms_col = find_col(df, ["de_aligned_forms","de_aligned_form","de_forms"], default="de_aligned_forms")
    de_lem_col = find_col(df, ["de_aligned_lemmas","de_aligned_lemma","de_lemmas"], default="de_aligned_lemmas")
    en_sent_col = find_col(df, ["sentence","en_sentence","en_sent"], default="sentence")
    de_sent_col = find_col(df, ["de_sentence","de_sent"], default="de_sentence")
    domain_col = find_col(df, ["Domain","domain"], default="Domain")

    # unified German label (prefer forms, fallback to lemmas)
    def best_de_label(row):
        forms = str(row.get(de_forms_col, "")).strip()
        if forms:
            return forms
        lem = str(row.get(de_lem_col, "")).strip()
        if lem:
            return lem
        return ""

    df["de_label"] = df.apply(best_de_label, axis=1)
    # normalize en_form, sense, domain
    df["en_form_norm"] = df[token_col].fillna("").astype(str).str.strip()
    df["sense_raw"] = df[sense_col].fillna("").astype(str)
    df["sense_display"] = df["sense_raw"].apply(normalize_sense)
    df["domain_norm"] = df[domain_col].fillna("").astype(str).str.strip()

    # precompute totals per (en_form, domain)
    total_per_form_domain = df.groupby(["en_form_norm", "domain_norm"]).size().to_dict()

    results = []

    # distinct (en_form, domain) pairs present in data
    pairs = sorted(df[["en_form_norm", "domain_norm"]].drop_duplicates().values.tolist(),
                   key=lambda x: (x[0].lower(), x[1].lower()))

    for en_form, domain in pairs:
        if not en_form:
            continue
        total_in_domain = int(total_per_form_domain.get((en_form, domain), 0))

        for sense in CANONICAL_SENSES:
            # select rows matching this (en_form, domain, sense)
            mask = (
                (df["en_form_norm"] == en_form) &
                (df["domain_norm"] == domain) &
                (df["sense_display"] == sense)
            )
            group = df.loc[mask]

            if len(group) == 0:
                # produce zero row for missing sense
                de_spans = ""
                de_counts = ""
                example_en = ""
                example_de = ""
                n_instances = 0
                pct_of_form = 0.0
            else:
                labels = list(group["de_label"].astype(str).map(str.strip))
                counts = Counter([lbl for lbl in labels if lbl])  # exclude empty labels from counts
                sorted_labels = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
                de_spans = ", ".join([lbl for lbl, cnt in sorted_labels])
                de_counts = ", ".join([f"{lbl}:{cnt}" for lbl, cnt in sorted_labels])
                example_en, example_de = choose_example(group, en_sent_col, de_sent_col)
                example_en = example_en if example_en is not None else ""
                example_de = example_de if example_de is not None else ""
                n_instances = len(group)
                pct_of_form = round(n_instances / total_in_domain, 4) if total_in_domain > 0 else 0.0

            results.append({
                "en_form": en_form,
                "domain": domain,
                "en_sense": sense,
                "de_aligned_forms": de_spans,
                "de_aligned_counts": de_counts,
                "example_en_sentence": example_en,
                "example_de_sentence": example_de,
                "n_instances": n_instances,
                "total_en_form_instances_in_domain": total_in_domain,
                "pct_of_form_in_domain": pct_of_form
            })

    out_df = pd.DataFrame(results, columns=[
        "en_form","domain","en_sense","de_aligned_forms","de_aligned_counts",
        "example_en_sentence","example_de_sentence",
        "n_instances","total_en_form_instances_in_domain","pct_of_form_in_domain"
    ])

    out_df.to_csv(output_csv, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)
    print(f"Wrote {len(out_df)} rows to {output_csv}")

    # quick check: show top 5 (en_form, domain) by total occurrences
    top_pairs = sorted(total_per_form_domain.items(), key=lambda x: (-x[1], x[0][0]))[:5]
    if top_pairs:
        print("Top (en_form, domain) pairs by occurrences (sample):")
        for (form, dom), cnt in top_pairs:
            print(f"  {form} @ {dom}: {cnt}")

if __name__ == "__main__":
    main()


Wrote 7215 rows to outputs/all_adjectives_en_de_aggregated_by_domain.csv
Top (en_form, domain) pairs by occurrences (sample):
  able @ Tech: 32
  better @ Phi: 28
  able @ Sci: 26
  able @ Med: 22
  right @ Phi: 19


# Top 20


In [19]:
#!/usr/bin/env python3
"""
aggregate_adjectives_by_sense_by_domain_top20.py

Input (default):
  outputs/all_adjectives_en_de_translations.csv

Output (default):
  outputs/all_adjectives_en_de_aggregated_by_domain_top20.csv

For each domain, selects the top-20 English adjectives by total occurrences
within that domain and writes one row per (en_form, domain, sense) for
the canonical senses: Affect, Judgement, Appreciation.
"""
import os
import csv
import pandas as pd
from collections import Counter, defaultdict

INPUT_CSV = "outputs/all_adjectives_en_de_translations.csv"
OUTPUT_CSV = "outputs/all_adjectives_en_de_aggregated_by_domain_top20.csv"

CANONICAL_SENSES = ["Affect", "Judgement", "Appreciation"]

def find_col(df, candidates, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    return default

def normalize_sense(s):
    if not s or pd.isna(s):
        return "(NULL)"
    s0 = str(s).strip().lower()
    # unify common variants
    if s0 in ("judgment", "judgement"):
        return "Judgement"
    if s0 == "affect":
        return "Affect"
    if s0 == "appreciation":
        return "Appreciation"
    # fallback: title case
    return s0.capitalize()

def choose_example(group, en_col, de_col):
    # prefer a row where de_col is non-empty, else any row
    for _, r in group.iterrows():
        if r.get(de_col) and str(r.get(de_col)).strip():
            return r.get(en_col, ""), r.get(de_col, "")
    if len(group) > 0:
        r = group.iloc[0]
        return r.get(en_col, ""), r.get(de_col, "")
    return "", ""

def main(input_csv=INPUT_CSV, output_csv=OUTPUT_CSV, top_k_per_domain=20):
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    df = pd.read_csv(input_csv, dtype=str).fillna("")

    # tolerant column lookup
    token_col = find_col(df, ["token_form","en_form","form","token"], default="token_form")
    sense_col = find_col(df, ["sense","en_sense"], default="sense")
    de_forms_col = find_col(df, ["de_aligned_forms","de_aligned_form","de_forms"], default="de_aligned_forms")
    de_lem_col = find_col(df, ["de_aligned_lemmas","de_aligned_lemma","de_lemmas"], default="de_aligned_lemmas")
    en_sent_col = find_col(df, ["sentence","en_sentence","en_sent"], default="sentence")
    de_sent_col = find_col(df, ["de_sentence","de_sent"], default="de_sentence")
    domain_col = find_col(df, ["Domain","domain"], default="Domain")

    # unified German label (prefer forms, fallback to lemmas)
    def best_de_label(row):
        forms = str(row.get(de_forms_col, "")).strip()
        if forms:
            return forms
        lem = str(row.get(de_lem_col, "")).strip()
        if lem:
            return lem
        return ""

    df["de_label"] = df.apply(best_de_label, axis=1)
    # normalize en_form, sense, domain
    df["en_form_norm"] = df[token_col].fillna("").astype(str).str.strip()
    df["sense_raw"] = df[sense_col].fillna("").astype(str)
    df["sense_display"] = df["sense_raw"].apply(normalize_sense)
    df["domain_norm"] = df[domain_col].fillna("").astype(str).str.strip()

    # compute totals per (en_form, domain)
    totals = df.groupby(["en_form_norm", "domain_norm"]).size().reset_index(name="count")

    # build top-K set per domain
    topk_per_domain = {}
    for domain, group in totals.groupby("domain_norm"):
        # sort by count desc then en_form asc to break ties deterministically
        sorted_rows = group.sort_values(by=["count", "en_form_norm"], ascending=[False, True])
        topk = sorted_rows.head(top_k_per_domain)["en_form_norm"].tolist()
        topk_per_domain[domain] = set(topk)

    results = []

    # distinct (en_form, domain) pairs to consider: only topk ones
    # We'll iterate over domains and their top-k forms to guarantee canonical order
    for domain in sorted(df["domain_norm"].unique()):
        top_forms = sorted(topk_per_domain.get(domain, []))
        if not top_forms:
            continue
        total_map = {row[0]: int(row[1]) for row in totals[totals["domain_norm"] == domain][["en_form_norm","count"]].values}

        for en_form in top_forms:
            total_in_domain = int(total_map.get(en_form, 0))

            for sense in CANONICAL_SENSES:
                mask = (
                    (df["en_form_norm"] == en_form) &
                    (df["domain_norm"] == domain) &
                    (df["sense_display"] == sense)
                )
                group = df.loc[mask]

                if len(group) == 0:
                    de_spans = ""
                    de_counts = ""
                    example_en = ""
                    example_de = ""
                    n_instances = 0
                    pct_of_form = 0.0
                else:
                    labels = list(group["de_label"].astype(str).map(str.strip))
                    counts = Counter([lbl for lbl in labels if lbl])  # exclude empty labels
                    sorted_labels = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
                    de_spans = ", ".join([lbl for lbl, cnt in sorted_labels])
                    de_counts = ", ".join([f"{lbl}:{cnt}" for lbl, cnt in sorted_labels])
                    example_en, example_de = choose_example(group, en_sent_col, de_sent_col)
                    example_en = example_en if example_en is not None else ""
                    example_de = example_de if example_de is not None else ""
                    n_instances = len(group)
                    pct_of_form = round(n_instances / total_in_domain, 4) if total_in_domain > 0 else 0.0

                results.append({
                    "en_form": en_form,
                    "domain": domain,
                    "en_sense": sense,
                    "de_aligned_forms": de_spans,
                    "de_aligned_counts": de_counts,
                    "example_en_sentence": example_en,
                    "example_de_sentence": example_de,
                    "n_instances": n_instances,
                    "total_en_form_instances_in_domain": total_in_domain,
                    "pct_of_form_in_domain": pct_of_form
                })

    out_df = pd.DataFrame(results, columns=[
        "en_form","domain","en_sense","de_aligned_forms","de_aligned_counts",
        "example_en_sentence","example_de_sentence",
        "n_instances","total_en_form_instances_in_domain","pct_of_form_in_domain"
    ])

    out_df.to_csv(output_csv, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)
    print(f"Wrote {len(out_df)} rows to {output_csv}")

    # Print a quick summary of selected top-k forms per domain
    print("\nSummary: top selected forms per domain (up to top {k}):".format(k=top_k_per_domain))
    for domain in sorted(topk_per_domain.keys()):
        tf = sorted(topk_per_domain[domain])
        print(f"  {domain}: {len(tf)} forms (sample: {tf[:10]})")

if __name__ == "__main__":
    main()


Wrote 660 rows to outputs/all_adjectives_en_de_aggregated_by_domain_top20.csv

Summary: top selected forms per domain (up to top 20):
  Art: 20 forms (sample: ['able', 'amazing', 'busy', 'cool', 'curious', 'dead', 'easy', 'free', 'funny', 'glad'])
  Bus: 20 forms (sample: ['able', 'available', 'better', 'blind', 'different', 'easy', 'fine', 'good', 'great', 'happier'])
  Edu: 20 forms (sample: ['able', 'afraid', 'available', 'bad', 'better', 'bored', 'brave', 'difficult', 'good', 'happy'])
  Ent: 20 forms (sample: ['able', 'amazing', 'available', 'better', 'capable', 'dead', 'different', 'enough', 'famous', 'good'])
  His: 20 forms (sample: ['able', 'about', 'alone', 'amazing', 'better', 'clear', 'dead', 'excited', 'good', 'great'])
  Med: 20 forms (sample: ['able', 'available', 'bad', 'better', 'clear', 'depressed', 'different', 'effective', 'good', 'happy'])
  Phi: 20 forms (sample: ['able', 'afraid', 'better', 'close', 'concerned', 'good', 'hard', 'important', 'interested', 'obvious

In [20]:

!zip -r outputs.zip outputs/

from google.colab import files
files.download("outputs.zip")


  adding: outputs/ (stored 0%)
  adding: outputs/all_adjectives_en_de_translations.csv (deflated 68%)
  adding: outputs/all_adjectives_en_de_aggregated_with_counts_overall.csv (deflated 62%)
  adding: outputs/all_adjectives_en_de_aggregated_by_domain.csv (deflated 65%)
  adding: outputs/all_adjectives_en_de_aggregated_by_domain_top20.csv (deflated 62%)
  adding: outputs/.ipynb_checkpoints/ (stored 0%)
  adding: outputs/all_adjectives_en_de_aggregated_with_counts_top20_overall.csv (deflated 60%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>